# classify attempts

In [ ]:
import os
from typing import List
from resolve_init import GetResolve

media_parent_path = '/Users/thom/Movies/Videos/Footages/tmp/素材'
proxy_parent_path = '/Users/thom/Movies/Videos/Footages/tmp/代理'


def get_subfolder_name(source_media_fullpath: List[str]):
    subfolders_name = []
    for i in source_media_fullpath:
        split_fullpath = i.split('/')
        subfolders_name.append(split_fullpath[-1])
    return subfolders_name


class Resolve:
    def __init__(self) -> None:
        self.resolve = GetResolve()
        self.project_manager = self.resolve.GetProjectManager()
        self.project = self.project_manager.GetCurrentProject()
        self.media_storage = self.resolve.GetMediaStorage()
        self.media_pool = self.project.GetMediaPool()
        self.root_folder = self.media_pool.GetRootFolder()
        # self.timeline = self.project.GetCurrentTimeline()
        self.media_fullpath_list = self.media_storage.GetSubFolderList(media_parent_path)

    def get_all_timeline(self) -> list:
        """
        Get all existing timelines.
        Return a list containing timeline object.
        """
        all_timeline = []
        for timeline_index in range(1, self.project.GetTimelineCount() + 1, 1):
            all_timeline.append(self.project.GetTimelineByIndex(timeline_index))
        return all_timeline

    def get_timeline_by_name(self, timeline_name: str):
        """Get timeline object by name."""
        all_timeline = self.get_all_timeline()
        timeline_dict = {timeline.GetName(): timeline for timeline in all_timeline}
        return timeline_dict.get(timeline_name, "")

    def get_subfolder_by_name(self, subfolder_name: str):
        """Get subfolders (folder object) under the root folder in the media pool."""
        all_subfolder = self.root_folder.GetSubFolderList()
        subfolder_dict = {subfolder.GetName(): subfolder for subfolder in all_subfolder}
        return subfolder_dict.get(subfolder_name, "")

In [5]:
from pprint import pprint
import os
from resolve_init import GetResolve

media_parent_path = '/Users/thom/Movies/Videos/Footages/tmp/素材'
proxy_parent_path = '/Users/thom/Movies/Videos/Footages/tmp/代理'

# Initialize Resolve native API
resolve = GetResolve()
project_manager = resolve.GetProjectManager()
project = project_manager.GetCurrentProject()
media_storage = resolve.GetMediaStorage()
media_pool = project.GetMediaPool()
root_folder = media_pool.GetRootFolder()

media_fullpath_list = media_storage.GetSubFolderList(media_parent_path)


def create_rendering_preset():
    proxy_preset = project.GetRenderPresetList()
    if len(proxy_preset) < 32:
        project.SetCurrentRenderMode(0)
        project.SetCurrentRenderFormatAndCodec("mov", "H265")
        rendering_setting = {
            "TargetDir": proxy_parent_path,
            "ExportVideo": True,
            "ExportAudio": True,
            "VideoQuality": 5000,
            "EncodingProfile": "Main",
            "MultiPassEncode": False,
            "AudioCodec": "aac",
            "AudioBitDepth": 16,
            "AudioSampleRate": 48000,
            "ColorSpaceTag": "Same as Project",
            "GammaTag": "Same as Project",
        }
        project.SetRenderSettings(rendering_setting)


create_rendering_preset()


# # Get render job status. completion percentage of the job.
# project.AddRenderJob()
# job_id_list = [render_job.get('JobId') for render_job in project.GetRenderJobList() ]
# project.GetRenderJobStatus(job_id_list[0])

def absolute_file_paths(directory) -> list:
    absolute_file_path_list = []
    for directory_path, _, filenames in os.walk(directory):
        for f in filenames:
            absolute_file_path_list.append(os.path.abspath(os.path.join(directory_path, f)))
    return absolute_file_path_list


INVALID_EXTENSION = ["DS_Store", "JPG", "JPEG", "SRT"]  # TODO, 小写的情况还待考虑进去

for cam_path in media_fullpath_list:
    filename_and_fullpath_dict = {os.path.split(os.path.splitext(path)[0])[1]: path for path in
                                  absolute_file_paths(cam_path) if path.split('.')[-1] not in INVALID_EXTENSION}

'DV#4/PRIVATE/M4ROOT/CLIP/C0004.MP4'